# Virtualization: Virtualization
_COSC 208, Introduction to Computer Systems, 2023-12-04_

## Announcements
* Project 4 due Thursday @ 11pm

## Virtual machines (VMs)

* Recall: _what is the purpose of an OS?_
    * Make computer hardware easy to use—e.g., an OS knows how to load an application's executable code from persistent storage (e.g., solid state drive (SSD)) into main memory, initialize the process's memory (code, heap, stack), and make the CPU execute the application's instructions
    * Support multiprocessing—i.e., running multiple applications simultaneously or concurrently
    * Allocate and manage hardware resources—e.g., decide when/which applications can use the CPU, decide when/which memory applications can use, prevent applications from stealing/accessing another application's CPU time or memory
* What is the purpose of a virtual machine?
    * Make computer hardware flexible to use
    * Support multiple OSes running concurrently
    * Allocate and manage hardware resources—e.g., decide when/which VMs can use the CPU, decide/when which memory VMs can use, prevent VMS from stealing/accessing another VM's CPU time or memory
* What is the motivation for using a virtual machine?
    * Run multiple OSes concurrently — _why would someone one to do this?_
        * Run software only supported by a specific OS
        * Software testing
        * Server consolidation
        * Security sandboxing
    * Flexibly use hardware resources

## Hypervisors

* _What additional software layer does a computer system need to support virtual machines?_ — hypervisor
* _Where does a hypervisor reside?_
    * Type 1: on top of hardware — e.g., KVM, Xen, Hyper-V, VMware vSphere
    * Type 2: on top of host OS — e.g., VirtualBox, VMware Workstation/Fusion
* _Why is a type 1 hypervisor better than a type 2 hypervisor?_
    * Resources can be oversubscribed
    * No overhead from host OS
* _Why is a type 2 hypervisor better than a type 1 hypervisor?_
    * Easy to use

* Q1: _Draw a diagram of a system that does **not** use a hypervisor or containers._

<p style="height:10em;"></p>

* Q2: _Draw a diagram of a system that uses a **type 1 hypervisor**._

<p style="height:10em;"></p>

* Q3: _Draw a diagram of a system that uses a **type 2 hypervisor**._

<p style="height:10em;"></p>

🛑 **STOP here** after completing the above questions; if you have extra time take a few deep breaths to reduce stress.

## CPU sharing

* Use same approach OS uses for sharing CPU between processes — Limited Direct Execution (LDE)
* _Recall: How does LDE work in OS?_
    * Put CPU in user mode
    * Give process exclusive access to CPU
    * Process issues a syscall to perform privileged operation
    * Timer interrupt allows OS to regain control
* To use LDE with a VM
    * Put CPU in user mode
    * Give VM exclusive access to CPU
    * Process issues a syscall -OR- OS attempts to perform privileged operation
    * Timer interrupt allows hypervisor to regain control
* Recall: _How does a process make a syscall?_ — load syscall number into register `x8` and execute `svc` instruction
* _What happens when a process makes a syscall?_
    1. CPU saves registers to the OS stack
    2. CPU switches to kernel mode
    3. CPU uses system call number to index into OS's table of syscall handlers 
    4. Branch and link to syscall handler code
    5. Restore registers from the OS stack
    6. Switch CPU to user mode
    7. Resume execution after `svc` 
* Syscalls with a type 1 hypervisor
    * Processes in virtualized OS do the same as before
    * Hypervisor’s syscall handler is invoked by hardware instead of OS’s syscall handler
    * Actually want to invoke OS's syscall handler — the hypervisor does this
    * Hypervisor knows where OS trap table is located, because populating syscall table is a privileged instruction, so hypervisor interposed on these operations when OS booted
* Steps when process in virtualized OS makes a syscall
    1. CPU saves registers to the hypervisor stack
    2. CPU switches to kernel mode
    3. CPU uses system call number to index into VM's table of syscall handlers 
    4. **Switch CPU from kernel to user mode**
    5. Branch and link to syscall handler code
    6. **Switch CPU from user to kernel mode**
    7. Restore registers from the hypervisor stack
    8. Switch CPU to user mode
    9. Resume execution after `svc`
* Extra work to always trap to hypervisor, which then invokes OS code
* Possible solutions
    * Hardware support — e.g., supervisor mode that is between user and kernel mode
    * Para-virtualization — modify the OS so it's aware there is a hypervisor and the two can interact directly
    * Containers — use a single OS, but create groups of processes that are isolated from other processes
        * OS's process abstraction already isolates each process's CPU time, memory, and registers, but processes can still use syscalls to pass messages to each other, wait for each other, access a common set of files, etc.

## Containers

* Containers — use a single OS, but create groups of processes that are isolated from other processes
    * OS's process abstraction already isolates each process's CPU time, memory, and registers, but processes can still use syscalls to pass messages to each other, wait for each other, access a common set of files, etc.
* Containers share the kernel — i.e., core OS functionality like syscalls, scheduler, I/O interface
    * VMs do **not** share the kernel
* Containers and VMs do **not** share the file system ==> containers and virtual machines do **not** share libraries, executables, etc.
* Containers and VMs do **not** share user accounts
* Containers and VMs do **not** share network stacks ==> e.g., each container or VM has its own Internet Protocol (IP) address
* Compute and storage resources (e.g., CPU, memory) available to containers and VMs can be limited
* Containers and VMs share the hardware